# Problem Formation

Given a Pattern String as an input, we want to know if it contains dark pattern in it. We use a balanced dataset cotaining all the instances in the Princeton dataset which are all dark patterns, and the instances in the 'normie.csv' file which are labeled as NOT dark patterns. Hence we have a balanced dataset consisting of pattern strings with dark pattern and without park patterns.

Then we use this labeled dataset to build and train supervised machine learning models, and select most suitable ones for our project.

----


In [1]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

# provides a simple way to both tokenize a collection of text documents and build a vocabulary of known words, but also to encode new documents using that vocabulary.
from sklearn.feature_extraction.text import CountVectorizer
# systematically compute word counts using CountVectorizer and them compute the Inverse Document Frequency (IDF) values and only then compute the Tf-idf scores.
from sklearn.feature_extraction.text import TfidfTransformer

# Bernoulli Naive Bayes (Similar as  MultinomialNB), this classifier is suitable for discrete data. The difference between MultinomialNB and BernoulliNB is that while  MultinomialNB works with occurrence counts, BernoulliNB is designed for binary/boolen features, which means in the case of text classification, word occurrence vectores (rather than word count vectors) may be more suitable to be used to train and use this classifier.
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

# Evaluation metrics
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score

# joblib is a set of tools to provide lightweight pipelining in Python. It provides utilities for saving and loading Python objects that make use of NumPy data structures, efficiently.
import joblib

import matplotlib.pyplot as plt
# import seaborn as sns

## Data Exploration

---
Import the merged dataset, and explore the dataset.

In [2]:
data = pd.read_csv('remove_more_noise.csv')

In [3]:
data.head(5)

,Pattern String,classification
0,FREE SHIPPING ON ORDERS OVER $100!,1
1,SOME EXCLUSIONS APPLY - LEARN MORE,1
2,HAVE A QUESTION? - CONTACT US,1
3,WELCOME TO 034MOTORSPORT!,1
4,SHOP AUDISHOP VOLKSWAGENPERFORMANCE SOFTWARE03...,1


---
`check the dataset information`

There are 2108 NOT NULL instances of pattern strings in the dataset.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2108 entries, 0 to 2107
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Pattern String  2108 non-null   object
 1   classification  2108 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 33.1+ KB


In [5]:
# check the distribution of the target value --- classification.

print('Distribution of the tags:\n{}'.format(data['classification'].value_counts()))

Distribution of the tags:
1    1301
0     807
Name: classification, dtype: int64


In [6]:
# Change the label into strings

data['classification'].replace({0:'Dark',1:'Not_Dark'}, inplace = True)

print(data.head(5))

print('\nDistribution of the tags:\n{}'.format(data['classification'].value_counts()))

                                      Pattern String classification
0                 FREE SHIPPING ON ORDERS OVER $100!       Not_Dark
1                 SOME EXCLUSIONS APPLY - LEARN MORE       Not_Dark
2                      HAVE A QUESTION? - CONTACT US       Not_Dark
3                          WELCOME TO 034MOTORSPORT!       Not_Dark
4  SHOP AUDISHOP VOLKSWAGENPERFORMANCE SOFTWARE03...       Not_Dark

Distribution of the tags:
Not_Dark    1301
Dark         807
Name: classification, dtype: int64


---
## Data Preparation

In [7]:
Y = data['classification']
X = data['Pattern String']

---
`Encode the target vales into integers` --- 'classification'

In [8]:
encoder = LabelEncoder()
encoder.fit(Y)
y = encoder.transform(Y)
y.shape

(2108,)

In [9]:
# check the mapping of encoding results (from 0 to 1 representing 'Dark', 'Not Dark')

list(encoder.classes_)

['Dark', 'Not_Dark']

In [10]:
# Check the frequency distribution of the training pattern classification with pattern classification names.

(unique, counts) = np.unique(Y, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[['Dark' 807]
 ['Not_Dark' 1301]]


In [11]:
# Check the frequency distribution of the encoded training pattern classification with encoded integers.

(unique, counts) = np.unique(y, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[   0  807]
 [   1 1301]]


---
`Encode the textual features into series of vector of numbers`

In [12]:
# First get the word count vector of the pattern string to encode the pattern string.

cv = CountVectorizer()
string_train_counts = cv.fit_transform(X)

# Then use the tf-idf score to transform the encoded word count pattern string vectors.

tfidf_tf = TfidfTransformer()
x = tfidf_tf.fit_transform(string_train_counts)

In [13]:
# save the CountVectorizer to disk

joblib.dump(cv, 'presence_CountVectorizer.joblib')

['presence_CountVectorizer.joblib']

---
# Rough Idea about the effect of different classifiers
---

In [14]:
# Five models are tested:
# -- Logistic Regression
# -- Linear Support Vector Machine
# -- Random Forest
# -- Multinomial Naive Bayes
# -- Bernoulli Naive Bayes
# -- KNN

classifiers = [LogisticRegression(), LinearSVC(), SVC(), RandomForestClassifier(), MultinomialNB(), BernoulliNB(), KNeighborsClassifier()]

In [15]:
# Calculate the accuracies of different classifiers using default settings.

acc = []
cm = []

for clf in classifiers:
    y_pred = cross_val_predict(clf, x, y, cv=5, n_jobs = -1)
    acc.append(metrics.accuracy_score(y, y_pred))
    cm.append(metrics.confusion_matrix(y, y_pred))

In [16]:
# List the accuracies of different classifiers.

for i in range(len(classifiers)):
    print(f"{classifiers[i]} accuracy: {acc[i]}")
    # print(f"Confusion Matris: {cm[i]}")

LogisticRegression() accuracy: 0.8420303605313093
LinearSVC() accuracy: 0.889943074003795
SVC() accuracy: 0.8709677419354839
RandomForestClassifier() accuracy: 0.9018026565464896
MultinomialNB() accuracy: 0.8851992409867173
BernoulliNB() accuracy: 0.8918406072106262
KNeighborsClassifier() accuracy: 0.8277988614800759


---
# Bernoulli Naive Bayes Classifier


---
### `Use default setting of classifier hyperparameters`

In [17]:
clf_bnb = BernoulliNB()

In [18]:
y_pred = cross_val_predict(clf_bnb, x, y, cv=5, n_jobs = -1)

In [19]:
clf_bnb.get_params()

{'alpha': 1.0, 'binarize': 0.0, 'class_prior': None, 'fit_prior': True}

---
`use the default setting of hyperparameters of the Bernoulli Naive Bayes classifier`

In [20]:
print("Accuracy:", metrics.accuracy_score(y, y_pred))
print("Confusion Matrix:\n", metrics.confusion_matrix(y, y_pred))

Accuracy: 0.8918406072106262
Confusion Matrix:
 [[ 667  140]
 [  88 1213]]


In [21]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   0,  755],
       [   1, 1353]])

---
### `Parameter Tunning of BernoulliNB classifier`
`Define the combination of parameters to be considered`

In [22]:
param_grid = {'alpha':[0,1], 
              'fit_prior':[True, False]}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [23]:
gs = GridSearchCV(clf_bnb,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [24]:
best_bnb = gs.fit(x,y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.0s finished


In [25]:
scores_df = pd.DataFrame(best_bnb.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_alpha', 'param_fit_prior']]

,rank_test_score,mean_test_score,param_alpha,param_fit_prior
0,1,0.891897,1,True
1,2,0.887620,1,False
2,3,0.865801,0,True
3,4,0.849673,0,False


In [26]:
best_bnb.best_params_

{'alpha': 1, 'fit_prior': True}

---
`Save the best BernoulliNB model for future use`

In [27]:
# save the model to local disk

joblib.dump(best_bnb, 'bnb_presence_classifier.joblib')

['bnb_presence_classifier.joblib']

---
# Random Forest Classifier


---
### `Use default setting of classifier hyperparameters`

In [39]:
clf_rf = RandomForestClassifier()

In [40]:
y_pred = cross_val_predict(clf_rf, x, y, cv=5, n_jobs = -1)

In [41]:
clf_rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

---
`use the default setting of hyperparameters of the Random Forest classifier.`

In [42]:
print("Accuracy:", metrics.accuracy_score(y, y_pred))
print("Confusion Matrix:\n", metrics.confusion_matrix(y, y_pred))

Accuracy: 0.8970588235294118
Confusion Matrix:
 [[ 625  182]
 [  35 1266]]


In [43]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   0,  660],
       [   1, 1448]])

---
### `Parameter Tunning of Random Forest classifier`
`Define the combination of parameters to be considered`

In [44]:
param_grid = {'bootstrap':[True,False], 
              'criterion':['gini','entropy'],
              'max_depth':[10,20,30,40,50,60,70,80,90,100, None],
              'min_samples_leaf':[1,2,4],
              'min_samples_split':[2,5,10],
              'n_estimators':[100,200,300,400,500,600]}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [45]:
gs = GridSearchCV(clf_rf,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [46]:
best_rf = gs.fit(x,y)

Fitting 5 folds for each of 2376 candidates, totalling 11880 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed: 16.3min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 19.6min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed: 24.7min
[Parallel(n_jobs=-1)]: Done 6026 tasks      | elapsed: 30.2min
[Parallel(n_jobs=-1)]: Done 7176 tasks      | elapsed: 35.7min
[Parallel(n_jobs=-1)]: Done 8426 tasks      | elapsed: 45.6min
[Parallel(n_jobs=-1)]: Done 9776 tasks      | elapsed: 53.3min
[Parallel(n_jobs=-1)]: Done 11226 tasks      

In [47]:
scores_df = pd.DataFrame(best_rf.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_bootstrap', 'param_criterion','param_max_depth','param_min_samples_leaf','param_min_samples_split','param_n_estimators']]

,rank_test_score,mean_test_score,param_bootstrap,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_n_estimators
0,1,0.905669,True,entropy,100,1,10,300
1,2,0.905192,True,gini,70,1,10,200
2,3,0.904721,True,entropy,100,1,5,500
3,4,0.904720,True,entropy,90,1,5,200
4,5,0.904718,True,gini,90,1,10,500
...,...,...,...,...,...,...,...,...
2371,2372,0.766682,True,gini,10,1,10,100
2372,2373,0.764340,False,entropy,10,1,10,400
2373,2374,0.762437,False,entropy,10,2,2,100
2374,2375,0.760522,True,entropy,10,4,2,100


In [48]:
best_rf.best_params_

{'bootstrap': True,
 'criterion': 'entropy',
 'max_depth': 100,
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 300}

---
`Save the best Random Forest model for future use`

In [49]:
# save the model to local disk

joblib.dump(best_rf, 'rf_presence_classifier.joblib')

['rf_presence_classifier.joblib']

---
# SVM Classifier


---
### `Use default setting of classifier hyperparameters`

In [28]:
clf_svm = LinearSVC()

In [29]:
y_pred = cross_val_predict(clf_svm, x, y, cv=5, n_jobs = -1)

In [30]:
clf_svm.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': True,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}

---
`use the default setting of hyperparameters of the Random Forest classifier.`

In [31]:
print("Accuracy:", metrics.accuracy_score(y, y_pred))
print("Confusion Matrix:\n", metrics.confusion_matrix(y, y_pred))

Accuracy: 0.889943074003795
Confusion Matrix:
 [[ 602  205]
 [  27 1274]]


In [32]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   0,  629],
       [   1, 1479]])

---
### `Parameter Tunning of SVM classifier`
`Define the combination of parameters to be considered`

In [33]:
param_grid = {'C':[0.1,1,10,100],
              'penalty':['l1','l2']}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [34]:
gs = GridSearchCV(clf_svm,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [35]:
best_svm = gs.fit(x,y)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.1s finished
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [36]:
scores_df = pd.DataFrame(best_svm.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_penalty', 'param_C']]

,rank_test_score,mean_test_score,param_penalty,param_C
0,1,0.899976,l2,100
1,2,0.899030,l2,10
2,3,0.890029,l2,1
3,4,0.845007,l2,0.1
4,5,NaN,l1,0.1
5,6,NaN,l1,1
6,7,NaN,l1,10
7,8,NaN,l1,100


In [37]:
best_svm.best_params_

{'C': 100, 'penalty': 'l2'}

---
`Save the best SVM model for future use`

In [38]:
# save the model to local disk

joblib.dump(best_svm, 'svm_presence_classifier.joblib')

['svm_presence_classifier.joblib']